In [12]:
%pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.


In [13]:
%pip install requests

In [14]:
import requests
import json


auth = json.load(open('secret.json'))['hf_auth']
headers = {"Authorization": auth}
API_URL = "https://api-inference.huggingface.co/models/google-t5/t5-large"

data = {
    "inputs": "summarize: Hello, world! Say it back to me please I want to hear hello world from AI",  # bitan format
    "parameters": {
        "max_new_tokens": 100,
        "min_length": 10,
        "temperature": 0.6,
        "top_p": 0.9,
        "num_beams": 3,
        "length_penalty": 0.4,
        "do_sample": True,
        "use_cache": True,
        "early_stopping": True
    }
}

response = requests.post(API_URL, headers=headers, json=data)
print(json.dumps(response.json(), indent=4))

# rip

# {
#     "error": "Model google-t5/t5-large is currently loading",
#     "estimated_time": 118.0269775390625
# }

[
    {
        "translation_text": "i want to hear hello world from ai"
    }
]


In [22]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, T5ForConditionalGeneration
import time

# Zbog siromastva i manjka vremena promptujemo online modele nekad

class ABCEncoderDecoderModel:
    def prompt(self, input, parameters) -> str:
        pass

class APIEncoderDecoderModel(ABCEncoderDecoderModel):
    def __init__(self, model_name="google-t5/t5-large"):
        """google-t5/t5-large  or google-t5/t5-3b"""
        self.model_name = model_name

    def prompt(self, input, parameters=None) -> str:
        if parameters is None:
            parameters = {
                "max_new_tokens": 200,
                "min_length": 10,
                "temperature": 0.6,
                "top_p": 0.9,
                "num_beams": 3,
                "length_penalty": 0.4,
                "do_sample": True,
                "use_cache": True,
                "early_stopping": True
            }
        
        auth = json.load(open('secret.json'))['hf_auth']
        headers = {"Authorization": auth}
        API_URL = f"https://api-inference.huggingface.co/models/{self.model_name}"

        data = {
            "inputs": input,
            "parameters": parameters # {
                # "max_new_tokens": 20,
                # "min_length": 10,
                # "temperature": 0.6,
                # "top_p": 0.9,
                # "num_beams": 3,
                # "length_penalty": 0.4,
                # "do_sample": True,
                # "use_cache": True,
                # "early_stopping": True
            #}
        }

        for _ in range(4):
            response = requests.post(API_URL, headers=headers, json=data)
            if "error" not in response:
                # print(response.json()[0])
                return response.json()[0]['translation_text']
            time.sleep(3)
        return "<ERROR>"

class LocalEncoderDecoderModel(ABCEncoderDecoderModel):
    def __init__(self, model_name="t5-large"):
        """t5-large or t5-3b"""

        tokenizer = AutoTokenizer.from_pretrained(model_name)
        device = "cuda" if torch.cuda.is_available() else "cpu"
        # device = "cpu"

        print(f"Instancing local pretrained model {model_name}")
        model = T5ForConditionalGeneration.from_pretrained(
            model_name,
            torch_dtype = torch.float16
        ).to(device)

        # device = torch.device("cuda:0")

        # Get allocated, reserved, and total memory on this GPU
        allocated_memory = torch.cuda.memory_allocated(device)
        reserved_memory = torch.cuda.memory_reserved(device)
        max_memory = torch.cuda.get_device_properties(device).total_memory

        print(f"Allocated: {allocated_memory / (1024**2):.2f} MB")
        print(f"Reserved:  {reserved_memory / (1024**2):.2f} MB")
        print(f"Total:     {max_memory / (1024**2):.2f} MB")
        print(model.device)

        self.tokenizer = tokenizer
        self.model = model

    def prompt(self, input, parameters=None) -> str:

        if parameters is None:
            parameters = {
                "max_new_tokens": 200,
                "min_length": 10,
                "temperature": 0.6,
                "top_p": 0.9,
                "num_beams": 3,
                "length_penalty": 0.4,
                "do_sample": True,
                "use_cache": True,
                "early_stopping": True
            }
        
        inputs_t = self.tokenizer(input, return_tensors="pt", truncation=True).to(self.model.device)
        # outputs = model.generate(**inputs, max_new_tokens=500)
        outputs = self.model.generate(
            **inputs_t,
            **parameters
        )

        out = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        del inputs_t
        torch.cuda.empty_cache()
        return out
    
    def free(self):
        try:
            del self.model
        except:
            pass

        try:
            del self.tokenizer
        except:
            pass
        torch.cuda.empty_cache()
        torch.cuda.synchronize()

In [31]:
class ABCSummaryAgent(ABCEncoderDecoderModel):
    pass

class APISummaryAgent(APIEncoderDecoderModel):
    def prompt(self, input, parameters=None) -> str:
        return super().prompt("summarize: " + input, parameters)

class LocalSummaryAgent(LocalEncoderDecoderModel):
    def prompt(self, input, parameters=None) -> str:
        return super().prompt("summarize: " + input, parameters)

In [32]:
cat_paragraph = "summarize: The domestic cat is a member of the Felidae, a family that had a common ancestor about 10 to 15 million years ago.[26] The evolutionary radiation of the Felidae began in Asia during the Miocene around 8.38 to 14.45 million years ago.[27] Analysis of mitochondrial DNA of all Felidae species indicates a radiation at 6.46 to 16.76 million years ago.[28] The genus Felis genetically diverged from other Felidae around 6 to 7 million years ago.[27] Results of phylogenetic research shows that the wild members of this genus evolved through sympatric or parapatric speciation, whereas the domestic cat evolved through artificial selection.[29] The domestic cat and its closest wild ancestor are diploid and both possess 38 chromosomes[30] and roughly 20,000 genes.[31]"

In [17]:
api_model = APIEncoderDecoderModel()
print(api_model.prompt(cat_paragraph))

the domestic cat is a member of the Felidae, a family that had a common ancestor about 10 to 15 million years ago.[26] .


In [18]:
lmodel = LocalEncoderDecoderModel()
print("init done")
print(lmodel.prompt(cat_paragraph))

Instancing local pretrained model t5-large


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Allocated: 3590.11 MB
Reserved:  3606.00 MB
Total:     8187.50 MB
cuda:0
init done
the domestic cat is a member of the Felidae, a family that had a common ancestor . the domestic cat and its closest wild ancestor are diploid and both possess 38 chromosomes[30] and roughly 20,000 genes .


In [23]:
lmodel.free()

In [33]:
cat_paragraph = "The domestic cat is a member of the Felidae, a family that had a common ancestor about 10 to 15 million years ago.[26] The evolutionary radiation of the Felidae began in Asia during the Miocene around 8.38 to 14.45 million years ago.[27] Analysis of mitochondrial DNA of all Felidae species indicates a radiation at 6.46 to 16.76 million years ago.[28] The genus Felis genetically diverged from other Felidae around 6 to 7 million years ago.[27] Results of phylogenetic research shows that the wild members of this genus evolved through sympatric or parapatric speciation, whereas the domestic cat evolved through artificial selection.[29] The domestic cat and its closest wild ancestor are diploid and both possess 38 chromosomes[30] and roughly 20,000 genes.[31]"

In [34]:
print(APISummaryAgent().prompt(cat_paragraph))

the domestic cat is a member of the Felidae, a family that had a common ancestor about 10 to 15 million years ago.[26] .


In [35]:
!nvidia-smi

Wed Jan  1 20:19:30 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 566.36                 Driver Version: 566.36         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4060 ...  WDDM  |   00000000:01:00.0  On |                  N/A |
| N/A   56C    P8              3W /  125W |     770MiB /   8188MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----